In [1]:
from roheboam.engine.core.metrics.mean_average_precision import MeanAveragePrecision

/home/kevin/miniconda3/envs/roheboam-cuda/lib/python3.7/site-packages/contracts/library/miscellaneous_aliases.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  m_new_contract('Container', ist(collections.Container))


In [2]:
def get_real_annotations(table):
    res = dict()
    ids = table['ImageID'].values.astype(np.str)
    labels = table['LabelName'].values.astype(np.str)
    xmin = table['XMin'].values.astype(np.float32)
    xmax = table['XMax'].values.astype(np.float32)
    ymin = table['YMin'].values.astype(np.float32)
    ymax = table['YMax'].values.astype(np.float32)

    for i in range(len(ids)):
        id = ids[i]
        label = labels[i]
        if id not in res:
            res[id] = dict()
        if label not in res[id]:
            res[id][label] = []
        box = [xmin[i], ymin[i], xmax[i], ymax[i]]
        res[id][label].append(box)

    return res


def get_detections(table):
    res = dict()
    ids = table['ImageID'].values.astype(np.str)
    labels = table['LabelName'].values.astype(np.str)
    scores = table['Conf'].values.astype(np.float32)
    xmin = table['XMin'].values.astype(np.float32)
    xmax = table['XMax'].values.astype(np.float32)
    ymin = table['YMin'].values.astype(np.float32)
    ymax = table['YMax'].values.astype(np.float32)

    for i in range(len(ids)):
        id = ids[i]
        label = labels[i]
        if id not in res:
            res[id] = dict()
        if label not in res[id]:
            res[id][label] = []
        box = [xmin[i], ymin[i], xmax[i], ymax[i], scores[i]]
        res[id][label].append(box)

    return res


/home/kevin/miniconda3/envs/roheboam-cuda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
def get_detections_and_annotations_from_image_file_name(image_file_name):
    predicted_boxes = np.vstack(image_file_name_to_prediction_map[image_file_name].values())[:,:4]
    predicted_scores = np.vstack(image_file_name_to_prediction_map[image_file_name].values())[:,4]
    predicted_labels = []
    for label, boxes in image_file_name_to_prediction_map[image_file_name].items():
        predicted_labels.extend([label] * len(boxes))

    target_boxes = np.vstack(image_file_name_to_annotations_map[image_file_name].values())
    target_labels = []

    for label, boxes in image_file_name_to_annotations_map[image_file_name].items():
        target_labels.extend([label] * len(boxes))

    return np.array(predicted_boxes), np.array(predicted_labels), np.array(predicted_scores), np.array(target_boxes), np.array(target_labels)

In [4]:
import numpy as np
import pandas as pd
annotations = pd.read_csv("./annotations.csv")
predictions = pd.read_csv("./detections.csv")
unique_classes = annotations['LabelName'].unique().astype(np.str)
annotation_image_file_names = annotations['ImageID'].unique()
prediction_image_file_names = predictions['ImageID'].unique()
image_file_name_to_prediction_map = get_detections(predictions)
image_file_name_to_annotations_map = get_real_annotations(annotations)

In [5]:
mAP_no_nms = MeanAveragePrecision(iou_threshold=0.5, labels=unique_classes)
for image_file_name in annotation_image_file_names:
    predicted_boxes, predicted_labels, predicted_scores, target_boxes, target_labels = get_detections_and_annotations_from_image_file_name(image_file_name)
    mAP_no_nms([predicted_boxes], [predicted_labels], [predicted_scores], [target_boxes], [target_labels])
print(mAP_no_nms.mean_average_precision)

/home/kevin/miniconda3/envs/roheboam-cuda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  
/home/kevin/miniconda3/envs/roheboam-cuda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/kevin/miniconda3/envs/roheboam-cuda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in 

0.4924558245868219
